In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from riotwatcher import LolWatcher, ApiError,TftWatcher
import time
from tqdm import tqdm

In [2]:
api_key = "----------------KEY--------------"
lol_watcher = LolWatcher(api_key)
my_region = 'br1'

In [3]:
de_para_lanes = {0:"Top",
                 1:"Jungle",
                 2:"Mid",
                 3:"Adc",
                 4:"Sup",
                 5:"Top",
                 6:"Jungle",
                 7:"Mid",
                 8:"Adc",
                 9:"Sup",}


def get_player_infos(players_list,id):
    player_info = lol_watcher.summoner.by_id(my_region, id)
    players_list.append(player_info)

def get_matches(player_puu_id):
    return lol_watcher.match.matchlist_by_puuid(my_region,player_puu_id,count  = 100) # trocar para 100 depois

def matchs_colect(list_matchs, match):
    dicionario = {}
    partida = lol_watcher.match.by_id(my_region,match)
    dicionario["Partida"] = match
    dicionario["Tempo"] = partida["info"]["gameDuration"]/60 #Tempo dado em segundos
    dicionario["Blue Team Victory"] =partida["info"]["teams"][0]["win"]

    for i in range (len(partida['info']["participants"])):
        lane = de_para_lanes[i] #partida['info']["participants"][i]["teamPosition"]
        team_id = partida['info']["participants"][i]["teamId"]
        status_by_position_by_side(dicionario, lane, team_id, i, partida)
    list_matchs.append(dicionario)
    
def status_by_position_by_side(dict, lane, side_code, n, base):
    if side_code == 100:
        side = "blue"
    else:
        side = "red"
    dict[f"champion_id_{lane}_{side}_side"] = base['info']["participants"][n]["championId"]
    dict[f"champion_{lane}_{side}_side"] = base['info']["participants"][n]["championName"]
    dict[f"kill_{lane}_{side}_side"] = base['info']["participants"][n]["kills"]
    dict[f"death_{lane}_{side}_side"] = base['info']["participants"][n]["deaths"]
    dict[f"assist_{lane}_{side}_side"] = base['info']["participants"][n]["assists"]
    dict[f"damage_per_minute_{lane}_{side}_side"] = base['info']["participants"][n]['challenges']["damagePerMinute"]
    dict[f"gold_per_minute_{lane}_{side}_side"] = base['info']["participants"][n]['challenges']["goldPerMinute"]
    dict[f"vision_per_minute_{lane}_{side}_side"] = base['info']["participants"][n]['challenges']["visionScorePerMinute"]


In [5]:
def main(primeira_vez):
    if primeira_vez:

        players = lol_watcher.league.challenger_by_queue(my_region,'RANKED_SOLO_5x5')
        time.sleep(1.25)
        df = pd.DataFrame.from_dict(players["entries"])
        df.sort_values( by = "leaguePoints", ascending = False).reset_index(inplace= True,drop = True)
        df.to_excel("challenger_list.xlsx",index=False)
        players_details = [ ]
        for id in tqdm(df["summonerId"]):
            get_player_infos(players_details,id)
            time.sleep(1.25)
        players_details_df = pd.DataFrame.from_dict(players_details)
        players_details_df.to_excel("players_details.xlsx",index=False)

        match_list = []
        matchies_info = []
        for puu_id in tqdm(players_details_df['puuid']):
            matchies = get_matches(puu_id)
            time.sleep(1.25) # para não chegarmos no limite de requisições da API
            for match in matchies:
                if match not in match_list:
                    try:
                        matchs_colect(matchies_info, match)
                        match_list.append(match)
                        time.sleep(1.25)
                    except:
                        pass
        
        partidas_coletadas = pd.DataFrame (match_list, columns = ['Partidas_coletadas'])
        matchies_info_df = pd.DataFrame.from_dict(matchies_info)
        partidas_coletadas.to_excel("partidas_coletadas.xlsx",index=False)
        matchies_info_df.to_excel("partidas_detalhadas.xlsx",index=False)


    else:
        players_list_df = pd.read_excel("challenger_list.xlsx")
        players_details_df = pd.read_excel("players_details.xlsx")
        partidas_coletadas = pd.read_excel("partidas_coletadas.xlsx")
        matchies_info_df = pd.read_excel("partidas_detalhadas.xlsx")
        match_list = []
        matchies_info = []
        for puu_id in tqdm(players_details_df['puuid']):
            matchies = get_matches(puu_id)
            time.sleep(1.5) # para não chegarmos no limite de requisições da API
            for match in matchies:
                if match in list(matchies_info_df['Partida']):
                    break
                elif match not in match_list:
                    try:
                        matchs_colect(matchies_info, match)
                        time.sleep(1.5)
                    except:
                        pass
        novas_partidas_coletadas = pd.DataFrame (match_list, columns = ['Partidas_coletadas'])
        new_matchies_info_df = pd.DataFrame.from_dict(matchies_info)
        partidas = pd.concat([partidas_coletadas, novas_partidas_coletadas], ignore_index=True)
        partidas_detail = pd.concat([matchies_info_df, new_matchies_info_df], ignore_index=True)
        partidas.to_excel("partidas_coletadas.xlsx",index=False)
        partidas_detail.to_excel("partidas_detalhadas.xlsx",index=False)

In [6]:
main(False)

100%|██████████| 200/200 [3:08:15<00:00, 56.48s/it]   
